# Important- when generating a json file for a new dataset, it is needed to change the paths of: train_source, test_source, label_source (lines 50-52) to the paths of the new dataset.
# It is also important to change the dataset_name in line 48, so the new json file will be saved in the new dataset folder and not at the previous one.

In [1]:
import os
import shutil
from nnunetv2.dataset_conversion.generate_dataset_json import generate_dataset_json
from batchgenerators.utilities.file_and_folder_operations import maybe_mkdir_p, save_json, subfiles

def delete_existing_json(output_folder):
    json_path = os.path.join(output_folder, 'dataset.json')
    if os.path.exists(json_path):
        os.remove(json_path)
        print(f"Deleted existing JSON file at: {json_path}")

def create_dataset_json_and_prepare_data(dataset_name, train_source, test_source, label_source, spacing):
    nnUNet_raw = r'/argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/'
    nnUNet_preprocessed = r'/argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_preprocessed/'

    imagesTr_dir = os.path.join(nnUNet_raw, dataset_name, 'imagesTr')
    imagesTs_dir = os.path.join(nnUNet_raw, dataset_name, 'imagesTs')
    labelsTr_dir = os.path.join(nnUNet_raw, dataset_name, 'labelsTr')
    maybe_mkdir_p(imagesTr_dir)
    maybe_mkdir_p(imagesTs_dir)
    maybe_mkdir_p(labelsTr_dir)
    maybe_mkdir_p(os.path.join(nnUNet_preprocessed, dataset_name))

    # Delete existing dataset JSON if it exists
    delete_existing_json(os.path.join(nnUNet_raw, dataset_name))

    # Generate the dataset JSON
    generate_dataset_json(
        os.path.join(nnUNet_raw, dataset_name),
        {0: 'mauthner_cells'},
        {'background': 0, 'cell': 1},
        num_training_cases=len(subfiles(train_source, suffix='.tif', join=False)),
        file_ending='.tif',
        dataset_description="Segmentation of mauthner cells",
        license="CC-BY-SA 4.0",
        spacing=spacing
    )
    
    # Copy test images and generate JSON with spacing
    # Uncomment and adapt this block if needed
    # images_test = subfiles(test_source, suffix='.tif', sort=True, join=False)
    # for i, im in enumerate(images_test):
    #     target_name = f'test_image_{i+1:04d}'
    #     shutil.copy(os.path.join(test_source, im), os.path.join(imagesTs_dir, target_name + '_0000.tif'))
    #     save_json({'spacing': spacing}, os.path.join(imagesTs_dir, target_name + '.json'))

if __name__ == '__main__':
    dataset_name = 'Dataset030_1030GlyRM1'

    train_source = r"/argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/Dataset030_1030GlyRM1/imagesTr/"
    test_source = r"/argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/Dataset030_1030GlyRM1/imagesTs/"
    label_source = r"/argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/Dataset030_1030GlyRM1/labelsTr/"
    
    # Spacing definition
    spacing = (1, 0.126, 0.126)

    # Create dataset JSON and prepare data
    create_dataset_json_and_prepare_data(dataset_name, train_source, test_source, label_source, spacing)


Deleted existing JSON file at: /argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/Dataset030_1030GlyRM1/dataset.json


In [ ]:
nnUNetv2_plan_and_preprocess -d 30 --verify_dataset_integrity


In [ ]:
nnUNet_compile=f nnUNetv2_train 024 2d 0


In [ ]:
nnUNetv2_predict -i /argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/Dataset031_1030GlyRM1_noSplit/imagesTs/ -o /argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_results/Dataset030_1030GlyRM1/TrPlusTs_4_folds_250ep/ -d 030 -c 2d -f all


### this prompt is to test the train-set
#### make sure to create this folde: Tr_exam_250ep at the right dir
nnUNetv2_predict -i /argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/Dataset030_1030GlyRM1/imagesTr/ -o /argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_results/Dataset030_1030GlyRM1/Tr_exam_250ep/ -d 030 -c 2d -f 0